In [5]:
!pip install gdown -U
!pip install evaluate

  Using cached evaluate-0.4.2-py3-none-any.whl.metadata (9.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00a 0:00:01


In [6]:
import os
os.environ["WANDB_DISABLED"]="true"

In [7]:
!mkdir -p results_promise_docs_bert

In [8]:
!gdown '1zLzqY2h10TNiq9NDKl0cJBTWSfHv014Y'

Downloading...
From: https://drive.google.com/uc?id=1zLzqY2h10TNiq9NDKl0cJBTWSfHv014Y
To: /kaggle/working/NFR_PROMISE.csv
100%|██████████████████████████████████████| 81.8k/81.8k [00:00<00:00, 74.1MB/s]


Aqui empiezo el código igual al de colab

In [9]:
from datasets import load_dataset

In [10]:
FILE_REQ='/kaggle/working/NFR_PROMISE.csv'
data_req = load_dataset('csv', data_files=FILE_REQ, sep=';')
print(data_req['train']['NFR'][21])

def create_labels(row):
  return {'labels': 'F' if row['NFR'] == 'F' else 'NF', 'id_doc':row['Doc']}

data_req = data_req.map(create_labels)
data_req = data_req.class_encode_column('labels')
data_req = data_req.class_encode_column('id_doc')
print(set(data_req['train']['labels']))
data_req = data_req['train']
data_req.features

Generating train split: 0 examples [00:00, ? examples/s]

F


Map:   0%|          | 0/625 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/625 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/625 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/625 [00:00<?, ? examples/s]

{0, 1}


{'Num': Value(dtype='int64', id=None),
 'Doc': Value(dtype='int64', id=None),
 'Requisito': Value(dtype='string', id=None),
 'NFR': Value(dtype='string', id=None),
 'labels': ClassLabel(names=['F', 'NF'], id=None),
 'id_doc': ClassLabel(names=['1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5', '6', '7', '8', '9'], id=None)}

In [11]:
from transformers import AutoTokenizer
#BERT: model_name = 'google-bert/bert-base-cased'
#DEBERTA: model_name = 'microsoft/deberta-v3-base'
#ROBERTA: model_name = 'FacebookAI/roberta-base'
model_name = 'google-bert/bert-base-cased' # modificar con el modelo que queramos cada vez

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
def encode_text(row, tokenizer):
  row_encode = tokenizer(row['Requisito'], truncation=True)
  return row_encode


encode_data = data_req.map(encode_text, fn_kwargs={'tokenizer':tokenizer})
encode_data

from sklearn.model_selection import KFold
import numpy as np

folds = KFold(n_splits=10, shuffle=True)

splits = folds.split(np.zeros(15)) #en el promise solo hay 15 docs
test_sets = []
train_val_sets = []
i=1
for train_val_idxs, test_idxs in splits:
  print (i)
  print(test_idxs)
  print('*******')
  data_test = encode_data.filter(lambda r : r['id_doc'] in test_idxs)
  data_train = encode_data.filter(lambda r : r['id_doc'] in train_val_idxs)
  data_train_val = data_train.train_test_split(train_size=0.9, stratify_by_column='labels')
  print(data_test)
  print(data_train)
  print(data_test.num_rows + data_train.num_rows)
  print(encode_data.num_rows == (data_test.num_rows + data_train.num_rows))
  print("***")


  test_sets.append(data_test)
  train_val_sets.append(data_train_val)
  i = i + 1

print(test_sets[0])
print(train_val_sets[0]['test'])

Map:   0%|          | 0/625 [00:00<?, ? examples/s]

1
[2 7]
*******


Filter:   0%|          | 0/625 [00:00<?, ? examples/s]

Filter:   0%|          | 0/625 [00:00<?, ? examples/s]

Dataset({
    features: ['Num', 'Doc', 'Requisito', 'NFR', 'labels', 'id_doc', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 53
})
Dataset({
    features: ['Num', 'Doc', 'Requisito', 'NFR', 'labels', 'id_doc', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 572
})
625
True
***
2
[ 4 11]
*******


...

Filter:   0%|          | 0/625 [00:00<?, ? examples/s]

Dataset({
    features: ['Num', 'Doc', 'Requisito', 'NFR', 'labels', 'id_doc', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 16
})
Dataset({
    features: ['Num', 'Doc', 'Requisito', 'NFR', 'labels', 'id_doc', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 609
})
625
True
***
10
[14]
*******


Filter:   0%|          | 0/625 [00:00<?, ? examples/s]

Filter:   0%|          | 0/625 [00:00<?, ? examples/s]

Dataset({
    features: ['Num', 'Doc', 'Requisito', 'NFR', 'labels', 'id_doc', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 24
})
Dataset({
    features: ['Num', 'Doc', 'Requisito', 'NFR', 'labels', 'id_doc', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 601
})
625
True
***
Dataset({
    features: ['Num', 'Doc', 'Requisito', 'NFR', 'labels', 'id_doc', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 53
})
Dataset({
    features: ['Num', 'Doc', 'Requisito', 'NFR', 'labels', 'id_doc', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 58
})


In [14]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [15]:
import evaluate
import numpy as np

In [16]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return metric.compute(predictions=predictions, references=labels)

# load metric
metric_name = 'f1'
metric = evaluate.load(metric_name)

In [17]:
import glob
from transformers import TrainingArguments, Trainer
from sklearn.metrics import classification_report

lr_values =[2e-5, 5e-5]

for j in range(len(train_val_sets)):
    best_lr = -1
    best_f1 = -1

    # para escribir los resultados
    with open(f'results_promise_docs_bert/res_docs_bert_{j}.txt', 'w') as f:
    # NOTA: si no va, deshacer el tab extra del siguiente bucle for
      for i, lr in enumerate(lr_values):
        print("================================================")
        print("j = " + str(j) + " // lr = " + str(lr))
        print("================================================")
      # argumentos para el entrenamiento
        training_args = TrainingArguments(
            #output_dir="my_checkpoint"+str(j),
            output_dir=f"my_checkpoint_{i}",
            overwrite_output_dir = True,
            num_train_epochs=10,
            #para probar tanto promise como nuestro coger 10 epochs
            evaluation_strategy="epoch",
            save_strategy="epoch",
            save_total_limit = 1,
            load_best_model_at_end=True,
            logging_strategy='epoch',
            optim="adamw_torch",
            per_device_train_batch_size=32,
            #lo dejaremos en 32
            #per_device_eval_batch_size=32,
            learning_rate=lr,
            #probar tanto 2e-5 como 5e-5
            weight_decay=0.01,
        )

        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
        # declarar el "entrenador"
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_val_sets[j]['train'],
            eval_dataset=train_val_sets[j]['test'],
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )

        # realizar el entrenamiento
        trainer.train()

        out_pred = trainer.predict(data_train_val['test'])
        if out_pred.metrics['test_f1'] > best_f1:
          best_f1 = out_pred.metrics['test_f1']
          best_lr = i
      # cargar el mejor modelo y resultados para test
      best_dir = glob.glob(f"my_checkpoint_{best_lr}/checkpoint*")
      print(best_dir)
      model_best = AutoModelForSequenceClassification.from_pretrained(best_dir[0])
      trainer_best = Trainer(
            model=model_best,
            args=None,
            train_dataset=None,
            eval_dataset=None,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )

      test_data = test_sets[j]
      outputs_pred = trainer_best.predict(test_dataset=test_data)

      # model predictions
      predictions = np.argmax(outputs_pred.predictions, axis=1)

      #d_res = classification_report(test_data['labels'], predictions, digits=3, return_dict = True)
      d_res = classification_report(test_data['labels'], predictions, digits=3, output_dict=True)
      print(d_res)
      f.write("================== Best lr ================\n")
      f.write(str(best_lr))
      f.write("\n")
      f.write( "================== Resultados ================\n")
      f.write(str(d_res))

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be re

j = 0 // lr = 2e-05


A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Pl

Epoch,Training Loss,Validation Loss,F1
1,0.631500,0.456645,0.891892
2,0.403600,0.213971,0.969697
3,0.207200,0.131024,0.985075
4,0.125200,0.069256,1.000000
5,0.049500,0.089186,0.968750
6,0.022800,0.201812,0.942857
7,0.013100,0.108993,0.970588
8,0.010600,0.071838,0.969697
9,0.011100,0.081397,0.955224
10,0.008900,0.101915,0.956522


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be re

...

j = 9 // lr = 5e-05


Epoch,Training Loss,Validation Loss,F1
1,0.586700,0.378287,0.891566
2,0.257800,0.202839,0.957746
3,0.103800,0.241614,0.957746
4,0.047800,0.229133,0.942857
5,0.013300,0.233927,0.960000
6,0.008700,0.333723,0.931507
7,0.005600,0.360476,0.945946
8,0.005500,0.402329,0.931507
9,0.006000,0.392894,0.931507
10,0.004200,0.392532,0.931507


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


['my_checkpoint_0/checkpoint-68']


{'0': {'precision': 0.9, 'recall': 0.5625, 'f1-score': 0.6923076923076923, 'support': 16}, '1': {'precision': 0.5, 'recall': 0.875, 'f1-score': 0.6363636363636364, 'support': 8}, 'accuracy': 0.6666666666666666, 'macro avg': {'precision': 0.7, 'recall': 0.71875, 'f1-score': 0.6643356643356644, 'support': 24}, 'weighted avg': {'precision': 0.7666666666666666, 'recall': 0.6666666666666666, 'f1-score': 0.6736596736596736, 'support': 24}}


Aquí acaba el código de colab. Falta hacer el zip para poder descargar los resultados (y opcionalmente usar el bot para que me avise del final de la ejecución)

In [18]:
import requests

# Definir la URL base y los parámetros
url = "https://api.callmebot.com/whatsapp.php"
params = {
    "phone": "+34653508040",
    "apikey": "1512070",
    "text": "🤖¡Ejecución terminada! 🤖\nVe a echarle un vistazo ☝🤓"
}

# Enviar la solicitud GET
response = requests.get(url, params=params)

# Imprimir el estado de la respuesta
print(response.status_code)
print(response.text)

200
<p>Message to: +34653508040<p>Text to send: 🤖¡Ejecución terminada! 🤖%0AVe a echarle un vistazo ☝🤓<p><b>Message queued.</b> You will receive it in a few seconds.


In [19]:
!zip -r results_promise_docs_bert.zip results_promise_docs_bert
from IPython.display import FileLink 
FileLink(r'results_promise_docs_bert.zip')

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: results_promise_docs_bert/ (stored 0%)
  adding: results_promise_docs_bert/res_docs_bert_4.txt (deflated 56%)
  adding: results_promise_docs_bert/res_docs_bert_8.txt (deflated 67%)
  adding: results_promise_docs_bert/res_docs_bert_2.txt (deflated 58%)
  adding: results_promise_docs_bert/res_docs_bert_5.txt (deflated 60%)
  adding: results_promise_docs_bert/res_docs_bert_0.txt (deflated 60%)
  adding: results_promise_docs_bert/res_docs_bert_9.txt (deflated 64%)
  adding: results_promise_docs_bert/res_docs_bert_1.txt (deflated 58%)
  adding: results_promise_docs_bert/res_docs_bert_6.txt (deflated 65%)
  adding: results_promise_docs_bert/res_docs_bert_3.txt (deflated 56%)
  adding: results_promise_docs_bert/res_docs_bert_7.txt (deflated 59%)


/kaggle/working/results_promise_docs_bert.zip